In [1]:
import time
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from tqdm import tqdm

from selenium_stealth import stealth

from selenium.common import TimeoutException

In [2]:
key_word = input('Введите ключевое слово.')
if ' ' in key_word:
    key_word = key_word.replace(' ', '%20')

num_of_pages = int(input('Введите количество страниц.'))

start_link = 'https://openi.nlm.nih.gov/'

In [4]:
links_list = []

while len(links_list) < num_of_pages * 100:
    try:
        from selenium import webdriver
        
        options = Options()
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        chromedriver = 'chromedriver.exe'
        service = ChromeService(executable_path=chromedriver)
        webdriver = webdriver.Chrome(options=options, service=service)

        stealth(webdriver,
            languages=["en-US", "en"],
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

        time.sleep(3)
        for i in tqdm(range(1, num_of_pages + 1)):
            if i * 100 == 100:
                webdriver.get(start_link + f'gridquery?q={key_word}&it=xg&m={i}&n={i * 100}')
                time.sleep(2)
                if WebDriverWait(webdriver, 30).until(ec.visibility_of_element_located((By.CSS_SELECTOR, '#grid'))):
                    for j in range(1, 101):
                        links_list.append(webdriver.find_element(By.CSS_SELECTOR, '#grid').find_element(By.CSS_SELECTOR, f'#grid > a:nth-child({j})').get_attribute('href'))

            else:
                webdriver.get(start_link + f'gridquery?q={key_word}&it=xg&m={i * 100 - 100 + 1}&n={i * 100}')
                time.sleep(2)
                WebDriverWait(webdriver, 30).until(ec.visibility_of_element_located((By.CSS_SELECTOR, '#grid')))
                for k in range(1, 101):
                    links_list.append(webdriver.find_element(By.CSS_SELECTOR, '#grid').find_element(By.CSS_SELECTOR, f'#grid > a:nth-child({k})').get_attribute('href'))
        time.sleep(3)
        webdriver.quit()

    except TimeoutException:
        webdriver.quit()
        continue

100%|██████████| 1/1 [00:12<00:00, 12.41s/it]


In [6]:
article_links = []
count = 0
while len(links_list) > count:
# while len(article_links) < len(links_list2):
    try:
        from selenium import webdriver
        options = Options()
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        chromedriver = 'chromedriver.exe'
        service = ChromeService(executable_path=chromedriver)
        webdriver = webdriver.Chrome(options=options, service=service)

        stealth(webdriver,
            languages=["en-US", "en"],
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

        for link in tqdm(links_list):
            count += 1
            webdriver.get(link)
            time.sleep(0.5)
            webdriver.execute_script("window.scrollTo(0, 2080)")
            time.sleep(1)
            WebDriverWait(webdriver, 30).until(ec.visibility_of_element_located((By.LINK_TEXT, 'PubMed Central')))
            l = webdriver.find_element(By.LINK_TEXT, 'PubMed').get_attribute('href')
            time.sleep(1)
            if l not in article_links:
                article_links.append(l)
                time.sleep(1)

    except TimeoutException:
        webdriver.close()
        continue
webdriver.quit()

100%|██████████| 100/100 [06:23<00:00,  3.83s/it]


In [8]:
pic_links = []
pic_links_dict = {}
count = len(article_links)

while len(pic_links_dict) < count:
    try:
        from selenium import webdriver

        options = Options()
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        chromedriver = 'chromedriver.exe'
        service = ChromeService(executable_path=chromedriver)
        webdriver = webdriver.Chrome(options=options, service=service)

        stealth(webdriver,
            languages=["en-US", "en"],
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

        for link in tqdm(article_links):
            webdriver.get(link)
            time.sleep(0.5)
            webdriver.execute_script("window.scrollTo(0, 2080)")
            time.sleep(1)
            WebDriverWait(webdriver, 30).until(ec.visibility_of_element_located((By.CSS_SELECTOR, '#figures')))
            pics = webdriver.find_element(By.CSS_SELECTOR, '#slides-container').find_elements(By.CLASS_NAME, 'figure-item ')
            time.sleep(1)
            for pic in pics:
                pic_link = pic.find_element(By.TAG_NAME, 'a').get_attribute('href')
                pic_links.append(pic_link)
                pic_links_dict[link] = pic_links
            pic_links = []

    except TimeoutException:
        webdriver.close()
        continue

webdriver.quit()

100%|██████████| 79/79 [08:01<00:00,  6.10s/it]


In [ ]:
result_directory = 'result_pics'
result_directory_abs_path = os.path.abspath(result_directory)

if not os.path.exists(result_directory + '/'):
    os.makedirs(result_directory + '/')


for item in tqdm(pic_links_dict.items()):
    key_name = str(item[0]).split('/')[-1]
    if not os.path.exists(result_directory + '/' + key_name + '/'):
        os.makedirs(result_directory + '/' + key_name + '/')
    for value in item[1]:
        prefs = {
            "profile.default_content_settings.popups": 0,
            "download.default_directory" : fr"{result_directory_abs_path}" + rf"\{key_name}",
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
        }

        from selenium import webdriver

        options = Options()
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        options.add_experimental_option('prefs', prefs)

        chromedriver = 'chromedriver.exe'
        service = ChromeService(executable_path=chromedriver)
        webdriver = webdriver.Chrome(options=options, service=service)

        stealth(webdriver,
            languages=["en-US", "en"],
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
        try:
            webdriver.get(value)
            time.sleep(1)
            while webdriver.execute_script("return document.readyState") != "complete":
                time.sleep(1)
            while any(filename.endswith('.crdownload') for filename in os.listdir(result_directory + '/' + key_name + '/')):
                time.sleep(3)
        except TimeoutException:
            webdriver.refresh()
webdriver.quit()